# Importing Libraries 

In [2]:
import pandas as pd
import numpy as np
import math
# import time 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import gmplot
from __future__ import division
from __future__ import print_function
import requests
import json
import urllib
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from requests.structures import CaseInsensitiveDict
from IPython.display import display, HTML

import random

# Importing Data 

In [3]:
loca = pd.read_excel('Final_locality.xlsx')
ad = pd.read_excel('updated_ad.xlsx')

In [4]:

Bhopal_localities = loca.loc[(loca.City == 'Bhopal')& (loca.wrong_data != True)]
Bhopal_localities.drop(columns = 'Unnamed: 0',inplace = True)


# Initializing Random Sample Data of 40 rows

In [5]:
Bhopal_ad = ad.loc[(ad.LocalityId.isin(Bhopal_localities.Id).astype(int)) &(ad.difference >=0) &(ad.Error == False) ]
Sample_ad=Bhopal_ad.sample(n=40)

# Adding Pincode Column
for index, row in Sample_ad.iterrows():
    pin = loca[loca.Id == row['LocalityId']].pincode.values[0]
    Sample_ad.at[index,'Pincode'] = pin

# Variable with the Longitude and Latitude
Sample =Sample_ad.loc[:,['Id','Latitude','Longitude','Pincode']]
#Fixing Index of Sample
Sample.reset_index(drop=True, inplace=True)


# Adding Time Window in VRP 

In [14]:
def open_tm(vehicle_no=4,data=None):
    Sample_x = data[:]
    str_x =''
    for x, y in zip(Sample_x['Latitude'], Sample_x['Longitude']):
        str_x+=f'{y},{x};'
    str_x = str_x[:-1]
    
    # url_y gives duration matrix
    url_y = f"http://router.project-osrm.org/table/v1/driving/{str_x}"      
    resp = requests.post(url_y)
    data_y = json.loads(resp.text)
    lol = data_y['durations']
    for i in range(len(lol)):
        for j in range(len(lol[i])):
            lol[i][j]=math.ceil(lol[i][j]/60)
    data = {}     
    data['time_matrix'] = lol[:]
    data['num_vehicles'] = vehicle_no
    data['depot'] = 0
    data['Id'] = Sample.to_dict()['Id']

    #adding Random Time Interval:
    l =[]
    for i in range(len(lol)):
        while True:
            w = random.randrange(30,120)
            q = random.randrange(30,500)
            if w+q <= 540:
                break
        l.append((q,q+w))
    data['time_windows'] = l[:]
    return data

def random_time_window(number,start_time,end_time,time_interval):
    temp_list = [(0,0)]
    for x in range(number-1):
        i = random.randrange(start_time,end_time,time_interval)
        temp_list.append((i,i+time_interval))
    return temp_list[:]
        

In [7]:

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))




In [8]:
# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]


In [9]:
def get_cumul_data(solution, routing, dimension):
    cumul_data = []
    for route_nbr in range(routing.vehicles()):
        route_data = []
        index = routing.Start(route_nbr)
        dim_var = dimension.CumulVar(index)
        route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            dim_var = dimension.CumulVar(index)
            route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
        cumul_data.append(route_data)
    return cumul_data

In [10]:
"""Solve the VRP with time windows."""
    
def show_solution_time(vehicle_no,df,start=[],end=[]):
#     if len(start) !=vehicle_no or len(end) !=vehicle_no:
#         print('Length of start and end list must be equal to number of vehicles ')
#         print("Please Specify Start or End Points list or use \"0\" in place of list")
        
#         return None
    
    data = open_tm(vehicle_no,df)
    print(data)
#     data['starts'] = start[:]
#     data['ends']  = end[:]
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        700,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == data['depot']:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data['depot']
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data['time_windows'][depot_idx][0],
            data['time_windows'][depot_idx][1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No Solution')

In [11]:
# show_solution_time(4,Sample)

In [ ]:
"""Vehicles Routing Problem (VRP) with Time Windows."""

"""Solve the VRP with time windows."""
# Instantiate the data problem.
x = Sample[:]
data = open_tm(4,x)
print(data)
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                       data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Time Windows constraint.
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    15,  # allow waiting time
    540,  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    time)
time_dimension = routing.GetDimensionOrDie(time)
# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])

# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)

{'time_matrix': [[0, 7, 23, 16, 11, 19, 19, 17, 24, 26, 13, 10, 16, 15, 21, 21, 15, 24, 24, 18, 9, 21, 13, 22, 16, 25, 17, 9, 15, 13, 21, 16, 8, 12, 18, 14, 8, 17, 13, 23], [7, 0, 20, 15, 9, 17, 18, 15, 23, 25, 12, 9, 15, 11, 20, 20, 14, 23, 23, 17, 8, 19, 9, 21, 15, 24, 16, 6, 13, 12, 20, 15, 7, 10, 16, 11, 8, 16, 12, 22], [25, 21, 0, 25, 19, 9, 23, 18, 24, 25, 18, 16, 20, 14, 24, 25, 22, 28, 24, 22, 19, 25, 18, 26, 14, 24, 15, 17, 12, 17, 25, 18, 20, 16, 8, 12, 21, 15, 23, 23], [16, 14, 25, 0, 13, 20, 13, 14, 21, 22, 11, 11, 13, 20, 17, 17, 8, 20, 21, 14, 9, 16, 19, 16, 14, 21, 15, 17, 16, 10, 17, 13, 10, 13, 19, 18, 10, 16, 7, 20], [11, 8, 18, 13, 0, 12, 13, 11, 18, 20, 7, 4, 10, 13, 15, 15, 11, 18, 18, 12, 6, 14, 12, 16, 10, 19, 11, 11, 8, 7, 15, 10, 7, 5, 12, 11, 9, 11, 11, 17], [19, 17, 9, 19, 13, 0, 17, 12, 17, 19, 12, 10, 13, 11, 17, 19, 16, 22, 17, 16, 13, 18, 14, 19, 7, 18, 9, 12, 6, 11, 18, 11, 14, 10, 5, 7, 15, 9, 17, 16], [20, 18, 23, 13, 13, 17, 0, 9, 15, 16, 8, 11, 7, 18

In [ ]:
len(data['time_windows'])

In [ ]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))
